# Classification I: Logistic Regression

**OBJECTIVES**:

- Differentiate between *Regression* and *Classification* problem settings
- Connect Least Squares methods to Classification through Logistic Regression
- Interpret coefficients of the model in terms of probabilities
- Discuss performance of classification model in terms of accuracy
- Understand the effect of an imbalanced target class on model performance

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from scipy import stats

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer, load_digits, load_iris

ModuleNotFoundError: No module named 'seaborn'

### Our Motivating Example



In [ ]:
default = pd.read_csv('https://raw.githubusercontent.com/jfkoehler/nyu_bootcamp_fa24/refs/heads/main/data/Default.csv', index_col = 0)

In [ ]:
default.info()

In [ ]:
default.head(2)

### Visualizing Default by Continuous Features

In [ ]:
#scatterplot of balance vs. income colored by default status

In [ ]:
sns.scatterplot(data = default, x = 'balance', y = 'income', hue = 'default')

In [ ]:
#boxplots for balance and income by default
fig, ax = plt.subplots(1, 2, figsize = (20, 5))
sns.boxplot(data = default, x = 'default', y = 'balance', ax = ax[0])
sns.boxplot(data = default, x = 'default', y = 'income', ax = ax[1])

### Considering only `balance` as the predictor



In [ ]:
#create binary default column
default['binary_default'] = np.where(default['default'] == 'No', 0, 1)

In [ ]:
#scatter of Balance vs Default
sns.scatterplot(data = default, x = 'balance', y = 'binary_default')

##### PROBLEM

1. Build a `LinearRegression` model with balance as the predictor.
2. Interpret the $r^2$ score and $rmse$ (`root_mean_squared_error`) for your regressor.
3. Predict the default for balances: `[200, 1000, 1500, 2000, 2500, 3500]`.  Do these make sense?

In [ ]:
X = default[['balance']]
y = default['binary_default']

In [ ]:
#regplot


### The Sigmoid aka Logistic Function


$$y = \frac{1}{1 + e^{-x}}$$

In [ ]:
#define the logistic
def logistic(x): return 1/(1 + np.exp(-x))

In [ ]:
#domain
x = np.arange(-10, 10, .1)

In [ ]:
#plot it
plt.plot(x, logistic(x))

### Usage should seem familiar

Fit a `LogisticRegression` estimator from `sklearn` on the features:

```python 
X = default[['balance']]
y = default['binary_default']
```

In [ ]:
#instantiate


In [ ]:
#define X and y
X = default[['balance']]
y = default['default']

In [ ]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 22)

In [ ]:
#fit on the train


In [ ]:
#examine train and test scores


### Evaluating the Classifier

In `scikitlearn` the primary default evalution metric is **accuracy** or percent correct.  We still need to compare this to our baseline -- typically predicting the most frequently occurring class.  Further, you can investigate the mistakes made with each class by looking at the **Confusion Matrix**.  A quick visualization of this is had using the `ConfusionMatrixDisplay`.

In [29]:
#baseline -- most frequently occurring class


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
#from estimator


### Similarities to our earlier work

In [ ]:
#there is a coefficient


In [ ]:
#there is an intercept


### Where was the line?

The version of the logistic we have just developed is actually:

$$ y = \frac{e^{ax + b}}{1 + e^{ax + b}} $$

Its output represents probabilities of being labeled the positive class in our example.  This means that we can interpret the output of the above function using our parameters, remembering that we used the `balance` feature to predict `default`.

In [ ]:
def predictor(x):
    line = clf.coef_[0]*x + clf.intercept_
    return np.e**line/(1 + np.e**line)

In [ ]:
#predict 1000
predictor(1000)

In [ ]:
#predict 2000
predictor(2000)

In [ ]:
#estimator has this too
clf.predict_proba(np.array([[1000]]))

In [ ]:
clf.predict(np.array([[1000]]))

### Using Categorical Features

In [ ]:
default.head(2)

In [ ]:
default['student_binary'] = np.where(default.student == 'No', 0, 1)

In [ ]:
X = default[['student_binary']]

In [ ]:
#instantiate and fit
clf = LogisticRegression()
clf.fit(X, y)

In [ ]:
#performance
clf.score(X, y)

In [ ]:
#coefficients
clf.coef_

In [ ]:
#compare probabilities
clf.predict_proba(X)

### Using Multiple Features



In [ ]:
default.columns

In [ ]:
features = ['balance', 'income', 'student_binary']
X = default.loc[:, features]
y = default['binary_default']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=22)

In [ ]:
clf = LogisticRegression().fit(X_train, y_train)

In [ ]:
clf.score(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

**Predictions**:

- student: yes
- balance: 1,500 dollars
- income: 40,000 dollars

In [ ]:
ex1 = np.array([[1500, 40_000, 1]])
#predict probability


- student: no
- balance: 1,500 dollars
- income: 40,000 dollars

In [ ]:
ex2 = np.array([[1500, 40_000, 0]])
#predict probability


### This is similar to our multicollinearity in regression; we will call it confounding

<center>
<img src = 'https://github.com/jfkoehler/nyu_bootcamp_fa24/blob/main/images/default_confound.png?raw=true' />
</center>

#### Using `scikitlearn` and its `Pipeline`

From the original data, to build a model involved:

1. One hot or dummy encoding the categorical feature.
2. Standard Scaling the continuous features
3. Building Logistic model

we can accomplish this all with the `Pipeline`, where the first step is a `make_column_transformer` and the second is a `LogisticRegression`.  

In [ ]:
from sklearn.pipeline import Pipeline 
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(default[['student', 'income', 'balance']], default['default'],
                                                   random_state = 22)

In [ ]:
# create OneHotEncoder instance
ohe = OneHotEncoder(drop = 'first')

In [ ]:
# create StandardScaler instance
sscaler = StandardScaler()

In [ ]:
# make column transformer
transformer = make_column_transformer((ohe, ['student']), 
                                     remainder = sscaler)

In [ ]:
# logistic regressor
clf = LogisticRegression()

In [ ]:
# pipeline
pipe = Pipeline([('transform', transformer), 
                 ('model', clf)])

In [ ]:
# fit it
pipe.fit(X_train, y_train)

In [ ]:
# score on train and test
print(f'Train Score: {pipe.score(X_train, y_train)}')
print(f'Test Score: {pipe.score(X_test, y_test)}')

In [ ]:
pipe.named_steps['model'].coef_

#### Problem

Below, a dataset on bank customer churn is loaded and displayed.  Your objective is to predict `Exited` or not.  Use `CreditScore`, `Gender`, `Age`, `Tenure`, and `Balance` as predictors.  Examine the confusion matrix display.  Was your classifier better at predicting exits or non-exits?

In [ ]:
from sklearn.datasets import fetch_openml

In [27]:
bank_churn = fetch_openml(data_id = 43390).frame

In [28]:
bank_churn.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1.0,15634602.0,Hargrave,619.0,France,Female,42.0,2.0,0.00,1.0,1.0,1.0,101348.88,1.0
1,2.0,15647311.0,Hill,608.0,Spain,Female,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58,0.0
2,3.0,15619304.0,Onio,502.0,France,Female,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57,1.0
3,4.0,15701354.0,Boni,699.0,France,Female,39.0,1.0,0.00,2.0,0.0,0.0,93826.63,0.0
4,5.0,15737888.0,Mitchell,850.0,Spain,Female,43.0,2.0,125510.82,1.0,1.0,1.0,79084.10,0.0


In [ ]:
#create train/test split -- random_state = 42


In [ ]:
#encode gender and standard scale other features


In [ ]:
#set up pipeline to encode/scale and then build logistic regression model


In [ ]:
#fit the model on training data


In [ ]:
#score on test and train


In [ ]:
#confusion matrix with test data


#### Compare to KNN and Grid Searching

Let's compare how this estimator performs compared to the `KNeighborsClassifier`.  This time however, we will be trying many KNN models across different numbers of neighbors.  One way we could do this is with a loop; something like:

```python
for neighbor in range(1, 30, 2):
    knn = KNeighborsClassifier(n_neighbors = neighbor).fit(X_train, y_train)
```



Instead, we can use the `GridSearchCV` object from sklearn.  This will take an estimator and a dictionary with parameters to be searched over.  

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# parameters we want to try
params = {'n_neighbors': range(1, 30, 2)}

In [ ]:
# estimator with parameters
knn = KNeighborsClassifier()

In [ ]:
# grid search object
grid = GridSearchCV(knn, param_grid=params)

In [ ]:
# fit it
X = default[['student_binary', 'income', 'balance']]
y = default['default']
grid.fit(X, y)

In [ ]:
# what was best?
grid.best_estimator_

In [ ]:
# score it 
grid.score(X, y)

#### Comparing Results

A good way to think about classifier performance is using a **confusion matrix**.  Below, we visualize this using the `ConfusionMatrixDisplay.from_estimator`. 

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
# a single confusion matrix
ConfusionMatrixDisplay.from_estimator(pipe, X_test, y_test, display_labels=['No', 'Yes'])

In [ ]:
# compare knn and logistic
fig, ax = plt.subplots(1, 2, figsize = (19, 5))
ConfusionMatrixDisplay.from_estimator(pipe, X_test, y_test, display_labels=['no', 'yes'], ax = ax[0])
ax[0].set_title('Logistic')
ConfusionMatrixDisplay.from_estimator(grid, X_test, y_test, display_labels=['no', 'yes'], ax = ax[1])
ax[1].set_title('KNN')

#### Practice

In [ ]:
from sklearn.datasets import load_breast_cancer

In [ ]:
cancer = load_breast_cancer(as_frame=True).frame

In [ ]:
cancer.head(3)

In [ ]:
# use all features


In [ ]:
# train/test split -- random_state = 42


In [ ]:
# pipeline to scale then knn


In [ ]:
# pipeline to scale then logistic


In [ ]:
# fit knn


In [ ]:
# fit logreg


In [ ]:
# compare confusion matrices on test data
